In [1]:
import pandas as pd
import sqlite3

In [2]:
volcanoes_df = pd.read_csv('../csv/cleaned/volcanoes_cleaned.csv')
volcanoes_df.columns = volcanoes_df.columns.str.replace(' ','_').str.lower()

In [3]:
eruptions_df = pd.read_csv('../csv/cleaned/eruptions_cleaned.csv')
eruptions_df.columns = eruptions_df.columns.str.replace(' ','_').str.lower()

In [4]:
# Connect to sqlite db and create tables if they don't exist
cnx = sqlite3.connect('volcanoes.db')
volcanoes_df.to_sql(name='volcanoes', con=cnx, if_exists='replace')
eruptions_df.to_sql(name='eruptions', con=cnx, if_exists='replace')

In [5]:
%%capture
%load_ext sql
%sql sqlite:///volcanoes.db

### All years in eruptions, considering both start_year and end_year

In [9]:
%%sql 
WITH all_years as ( SELECT 
    start_year as eruption_year
FROM
    (SELECT 
        start_year
    FROM
        eruptions
    WHERE
        start_year >= 1900) A
UNION (SELECT 
    end_year
FROM
    eruptions
WHERE
    end_year >= 1900) ORDER BY eruption_year
)
SELECT 
    eruption_year,
    SUM(start_year = ay.eruption_year) AS started_in_year_count,
    SUM(end_year = ay.eruption_year) AS ended_in_year_count,
    COUNT(*) AS total_eruptions
FROM
    all_years ay
        LEFT JOIN
    eruptions e ON ay.eruption_year BETWEEN e.start_year AND e.end_year
GROUP BY ay.eruption_year;

# Volcanoes with most eruptions since 1900 
SELECT 
	SUM(e.end_year-e.start_year+1),
    COUNT(e.eruption_number) eruptions_count,
    CONCAT(v.volcano_name, ' (', v.country, ')') as volcano_name
FROM
    eruptions e
        JOIN
    volcanoes v ON e.volcano_number = v.volcano_number
WHERE
    e.start_year > 1900
GROUP BY v.volcano_name, v.country
ORDER BY eruptions_count DESC
LIMIT 20;

 * sqlite:///volcanoes.db
(sqlite3.OperationalError) near "(": syntax error
[SQL: WITH all_years as ( SELECT 
    start_year as eruption_year
FROM
    (SELECT 
        start_year
    FROM
        eruptions
    WHERE
        start_year >= 1900) A
UNION (SELECT 
    end_year
FROM
    eruptions
WHERE
    end_year >= 1900) ORDER BY eruption_year
)
SELECT 
    eruption_year,
    SUM(start_year = ay.eruption_year) AS started_in_year_count,
    SUM(end_year = ay.eruption_year) AS ended_in_year_count,
    COUNT(*) AS total_eruptions
FROM
    all_years ay
        LEFT JOIN
    eruptions e ON ay.eruption_year BETWEEN e.start_year AND e.end_year
GROUP BY ay.eruption_year;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


### Volcanoes with most eruptions since 1900 


In [8]:
%%sql 
SELECT 
    COUNT(e.eruption_number) eruptions_count,
    v.volcano_name || ' (' || v.country || ')' as volcano_name
FROM
    eruptions e
        JOIN
    volcanoes v ON e.volcano_number = v.volcano_number
WHERE
    e.start_year > 1900
GROUP BY v.volcano_name, v.country
ORDER BY eruptions_count DESC
LIMIT 20;

 * sqlite:///volcanoes.db
Done.


eruptions_count,volcano_name
94,"Fournaise, Piton de la (France)"
83,Etna (Italy)
77,Asosan (Japan)
73,Klyuchevskoy (Russia)
57,Ruapehu (New Zealand)
53,Kilauea (United States)
52,Asamayama (Japan)
52,Bezymianny (Russia)
49,Raung (Indonesia)
48,Karangetang (Indonesia)
